In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
"""
This is the starter code and some suggested architecture we provide you with. 
But feel free to do any modifications as you wish or just completely ignore 
all of them and have your own implementations.
"""
# some inspirations were taken from 
# https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/

from collections import Counter

import numpy as np
from numpy import genfromtxt
import scipy.io
from scipy import stats

import random

np.random.seed(1242)
random.seed(1242)

class DecisionTree:

    def __init__(self,CostMethod,MaxDepth,Type,Features):
        self.CostMethod = CostMethod
        self.MaxDepth = MaxDepth
        self.Type = Type
        self.Features = Features
        """
        TODO: initialization of a decision tree
        """
        
    @staticmethod
    def entropy(y):
        """
        TODO: implement a method that calculates the entropy given all the labels
        """
        y = np.array(y)
        # assuming binary class
        if (len(y)==0):
            H = 0
        else:
            Pc1 = np.sum(y[y==1])/len(y)
            Pc0 = 1-Pc1
            if ((Pc1==0) |(Pc0==0)):
                H=0
            else:
                H = - (Pc1*np.log(Pc1)+Pc0*np.log(Pc0))
        return H

    @staticmethod
    def information_gain(X, y, thresh):
        """
        TODO: implement a method that calculates information gain given a vector of features
        and a split threshold
        """
        return 0

    @staticmethod
    def gini_impurity(y):
        """
        TODO: implement a method that calculates the gini impurity given all the labels
        """
        return 0

    @staticmethod
    def gini_purification(X, y, thresh):
        """
        TODO: implement a method that calculates reduction in impurity gain given a vector of features
        and a split threshold
        """
        return 0

    def split(self,SplitVal,SplitIndex, Data):
        """
        TODO: implement a method that return a split of the dataset given an index of the feature and
        a threshold for it
        """
        left,right = list(), list()
        for DataRow in Data:
            if (DataRow[SplitIndex]<=SplitVal):
                left.append(DataRow)
            else:
                right.append(DataRow)
        return left,right

    def segmenter(self, Node):
        """
        TODO: compute entropy gain for all single-dimension splits,
        return the feature and the threshold for the split that
        has maximum gain
        """
        X = np.array(Node['Data'])
        U,V = np.shape(X)
        #print(U,V)
        Left,Right = self.split(X[0,0],0,X)
        if (self.CostMethod =='Entropy'):
            Uleft = len(Left)
            if(Uleft ==0):
                Hleft =0
            else:
                Hleft = self.entropy(np.array(Left)[:,-1])
            Uright = len(Right)
            if(Uright ==0):
                Hright =0
            else:
                Hright = self.entropy(np.array(Right)[:,-1])
            Cost = (Uright*Hright+Uleft*Hleft)/U
        baselineVal,baselineFeatureIndex,baselineCost,baselineLeft,baselineRight=X[0,0],0,Cost,Left,Right
        if (self.Type=='Forest'):
            #print(V-1)
            FeatureNos = np.array(random.sample(range(0,V-1),int(np.sqrt(V-1))))
            
            #print(FeatureNos)
        else:

            FeatureNos = np.linspace(0,V-2,V-1,dtype='int32')
            #print(FeatureNos)
 
        for i in range(0,U):
            
            for j in range(0,len(FeatureNos)):

                Left,Right = self.split(X[i,FeatureNos[j]],FeatureNos[j],X)
                if (self.CostMethod =='Entropy'):
                    Uleft = len(Left)
                    if(Uleft ==0):
                        Hleft =0
                    else:
                        Hleft = self.entropy(np.array(Left)[:,-1])
                    Uright = len(Right)
                    if(Uright ==0):
                        Hright =0
                    else:
                        Hright = self.entropy(np.array(Right)[:,-1])
                    Cost = (Uright*Hright+Uleft*Hleft)/U
                if (Cost<baselineCost):
                    #print(X[i,FeatureNos[j]],FeatureNos[j])
                    baselineVal,baselineFeatureIndex,baselineCost,baselineLeft,baselineRight=X[i,FeatureNos[j]
                                                                                              ],FeatureNos[j],Cost,Left,Right

        Node['Left']   ={'Data':baselineLeft,
                         'Label':'Node',
                         'Threshold':'nan',
                         'Index' : 'nan',
                         'Left':'nan',
                         'Right' : 'nan',
                         'LeafLabel' : 'nan', 
                        }
        Node['Right']  ={'Data':baselineRight,
                         'Label':'Node',
                         'Threshold':'nan',
                         'Index' : 'nan',
                         'Left':'nan',
                         'Right' : 'nan',
                         'LeafLabel' : 'nan', 
                        }
        Node['Threshold'] = baselineVal
        Node['Index'] = baselineFeatureIndex
        return 
    def leaf(self,X):
        X['Label'] = 'Leaf'
        y = np.array(X['Data'])[:,-1]
        if (np.count_nonzero(y==1)>np.count_nonzero(y==0)):
            X['LeafLabel']=1
        else:
            X['LeafLabel']=0
        return
    def CreateBranch(self,X,DepthLevel):

        if((DepthLevel+1)>self.MaxDepth):

            self.leaf(X)
        elif((X['Left']!= 'nan') &(X['Right']!= 'nan') &((len(X['Left'])==0)|(len(X['Right'])==0))):

            self.leaf(X)
        else:

            self.segmenter(X)
            if(len(X['Left']['Data'])==0):  # see if there is no segmentation, if so then end the node here
                self.leaf(X['Right'])
                X['Left']['Label'] = 'Leaf'
                X['Left']['LeafLabel'] = X['Right']['LeafLabel']
            elif(len(X['Right']['Data'])==0):  # see if there is no segmentation, if so then end the node here
                self.leaf(X['Left'])
                X['Right']['Label'] = 'Leaf'
                X['Right']['LeafLabel'] = X['Left']['LeafLabel']
            else:
                print(DepthLevel,self.MaxDepth)
                self.CreateBranch(X['Left'],DepthLevel+1)
                self.CreateBranch(X['Right'],DepthLevel+1)
        return
    
    def fit(self, X):
        """
        TODO: fit the model to a training set. Think about what would be 
        your stopping criteria
        """
        NodeLevel =0
        
        Root ={'Data':X,
               'Label':'Node',
               'Threshold':'nan',
               'Index' : 'nan',
               'Left':'nan',
               'Right' : 'nan',
               'LeafLabel' : 'nan', 
               }
        #SplitVal,SplitIndex,LeftGroup,RightGroup = self.segmenter(X,NodeLevel)
        self.CreateBranch(Root,NodeLevel)
        
        #self.__repr__(Root,'\t') # if you want to visualize the tree
        
        return Root

    def predict(self, Node,X):
        """
        TODO: predict the labels for input data 
        """
        if(Node['Label']=='Leaf'):
            Label = Node['LeafLabel']
        else:
            y = Node['Index']
            if (X[y]>Node['Threshold']):
                Label = self.predict(Node['Right'],X)
            else:
                Label = self.predict(Node['Left'],X)
        return Label

    def __repr__(self,Node,tab):
        """
        TODO: one way to visualize the decision tree is to write out a __repr__ method
        that returns the string representation of a tree. Think about how to visualize 
        a tree structure. You might have seen this before in CS61A.
        """
        if(Node['Label']=='Node'):
            print(tab,'if X[',self.Features[Node['Index']],']>',Node['Threshold'])
            self.__repr__(Node['Right'],tab+'\t')
            print(tab,'else')
            self.__repr__(Node['Left'],tab+'\t')
        else:
            print(tab,'y=',Node['LeafLabel'])
        return 0

def DataSplit(data, labels, val_size):
    num_items = len(data)
    assert num_items == len(labels)
    assert val_size >= 0
    if val_size < 1.0:
        val_size = int(num_items * val_size)
    train_size = num_items - val_size
    idx = np.random.permutation(num_items)
    data_train1 = data[idx][:train_size]
    label_train = labels[idx][:train_size]
    data_val = data[idx][train_size:]
    label_val = labels[idx][train_size:]
    return data_train1, data_val, label_train, label_val
if __name__ == "__main__":
    #dataset = "titanic"
    dataset = "spam"
    Bagging = 'On'
    Validation = 'On' # if off that means we are doing the test data 
    ClassifierType = 'Forest'
    DataFileName = 'SPAMRFTestV1.mat'
    
    #FeatureName = []
    if dataset == "titanic":
        # Load titanic data       
        data1 =scipy.io.loadmat('TitanicProcessedV2.mat')
        X = data1["X"]
        y = data1["y"]
        Z = data1["Z"]    
        y = y.reshape(np.size(y),1)
        FeatureName =['pclass','sex','age','sibsp','parch','fare','embarked']# for titanic
        # TODO: preprocess titanic dataset
        # Notes: 
        # 1. Some data points are missing their labels
        # 2. Some features are not numerical but categorical
        # 3. Some values are missing for some features
        
    elif dataset == "spam":
        features = [
            "pain", "private", "bank", "money", "drug", "spam", "prescription",
            "creative", "height", "featured", "differ", "width", "other",
            "energy", "business", "message", "volumes", "revision", "path",
            "meter", "memo", "planning", "pleased", "record", "out",
            "semicolon", "dollar", "sharp", "exclamation", "parenthesis",
            "square_bracket", "ampersand"
        ]
        assert len(features) == 32
        FeatureName = features
        # Load spam data
        path_train = 'datasets/spam-dataset/spam_data.mat'
        data = scipy.io.loadmat(path_train)
        X = data['training_data']
        y = np.squeeze(data['training_labels'])
        Z = data['test_data']
        class_names = ["Ham", "Spam"]
         
    else:
        raise NotImplementedError("Dataset %s not handled" % dataset)
    # splitting data into training and validation
    if (Validation == 'On'):
        ValidationSize = int(len(y)*0.20)
        TrainingData,ValidationData,TrainingLabel,ValidationLabel = DataSplit(X,y,ValidationSize)
        # converting the data into array for easier preprocessing
        TrainingData = np.array(TrainingData)
        TrainingLabel = np.array(TrainingLabel)
        ValidationData = np.array(ValidationData)
        ValidationLabel = np.array(ValidationLabel)
    else:
        TrainingData = np.array(X)
        TrainingLabel = np.array(y)
        ValidationData = np.array(Z)
        ValidationSize = len(Z)
    #print(np.shape(TrainingData))
    TrainingData = np.concatenate((TrainingData,TrainingLabel.reshape(len(TrainingLabel),1)),axis=1)
    #print(np.shape(TrainingData))
    #return
    #creating an instance of the classifier
    if (Bagging=='Off'):
       
        if (Validation == 'On'):
            MaxDepth = np.linspace(7,7,1)
            ValidationError = np.zeros((len(MaxDepth),1),dtype ='float64')
            TrainingError = np.zeros((len(MaxDepth),1),dtype ='float64')
        else:
            MaxDepth = np.linspace(1,1,1)
        
        for j in range(0,len(MaxDepth)):
            print(MaxDepth[j])
            classifier = DecisionTree('Entropy',MaxDepth[j],ClassifierType,FeatureName)
            #print(j)
            import time
            start = time.time()
            #print(j)
            Root= classifier.fit(TrainingData)
            ValidationPredicted = np.zeros((ValidationSize,1),dtype ='int64')
            TrainingPredicted = np.zeros((len(TrainingLabel),1),dtype ='int64')
            for i in range(0,len(ValidationData)):
                ValidationPredicted[i] = classifier.predict(Root,ValidationData[i,:])
            for i in range(0,len(TrainingLabel)):
                TrainingPredicted[i] = classifier.predict(Root,TrainingData[i,:])
            if (Validation=='On'):
                ValidationError[j] = np.sum(np.abs((ValidationLabel.reshape(ValidationSize,1)
                                                    -ValidationPredicted.reshape(ValidationSize,1))
                                                   .reshape(ValidationSize,1)))/ValidationSize
                TrainingError[j] = np.sum(np.abs((TrainingLabel.reshape(len(TrainingLabel),1)
                                            -TrainingPredicted.reshape(len(TrainingLabel),1))
                                           .reshape(len(TrainingLabel),1)))/len(TrainingLabel)
                end = time.time()
                print(ValidationError,TrainingError)
                print(end-start,'seconds')
                scipy.io.savemat(DataFileName,{'ValidationError':ValidationError,'TrainingError':TrainingError,'Tree Depth':MaxDepth})
            else:
                flat_list = [item for sublist in ValidationPredicted for item in sublist]
                scipy.io.savemat(DataFileName,{'Tree':Root,'Prediction':flat_list})
                import save_csvTitanicDT
                save_csvTitanicDT.results_to_csv(flat_list)
    else:
        BagNos = 100
        BagSize = int(1*len(TrainingData))
        if (Validation == 'On'):
            MaxDepth = np.linspace(7,7,1)
            ValidationError = np.zeros((len(MaxDepth),1),dtype ='float64')
            TrainingError = np.zeros((len(MaxDepth),1),dtype ='float64')
        else:
            MaxDepth = np.linspace(5,5,1)
        for j in range(0,len(MaxDepth)): 
            print('Depth',MaxDepth[j])
            ValidationPredicted = np.zeros((ValidationSize,BagNos),dtype ='float64')
            TrainingPredicted = np.zeros((len(TrainingLabel),BagNos),dtype ='float64')
            for k in range(0,BagNos):
                print('Bag',k)
                np.random.seed(k)
                BagVariable=np.random.randint(BagSize,size=BagSize)
                TrainingBagData = TrainingData[BagVariable,:]
                classifier = DecisionTree('Entropy',MaxDepth[j],ClassifierType,FeatureName) # type either Forest or Tree
                #import time
                #start = time.time()
                #Level = 0 
                Root= classifier.fit(TrainingBagData)
                print(j+1)
                #ValidationPredicted = np.zeros((ValidationSize,1),dtype ='int64')
                for i in range(0,ValidationSize):
                    #print(i,k)
                    ValidationPredicted[i,k] = classifier.predict(Root,ValidationData[i,:])
                for i in range(0,len(TrainingLabel)):
                    TrainingPredicted[i] = classifier.predict(Root,TrainingData[i,:])
            ValidationPredicted = np.round(np.sum(ValidationPredicted,axis=1)/BagNos,decimals =0)
            TrainingPredicted = np.round(np.sum(TrainingPredicted,axis=1)/BagNos,decimals =0)
            if (Validation=='On'):
                ValidationError[j] = np.sum(np.abs((ValidationLabel.reshape(ValidationSize,1)
                                                     -ValidationPredicted.reshape(ValidationSize,1))
                                                     .reshape(ValidationSize,1)))/ValidationSize
                TrainingError[j] = np.sum(np.abs((TrainingLabel.reshape(len(TrainingLabel),1)
                                            -TrainingPredicted.reshape(len(TrainingLabel),1))
                                           .reshape(len(TrainingLabel),1)))/len(TrainingLabel)
                print(j,ValidationError[j])
                scipy.io.savemat(DataFileName,{'ValidationError':ValidationError,'TrainingError':TrainingError,'Tree Depth':MaxDepth,
                                                        'BagSize':BagSize,'BagNos':BagNos})
            
            else:
                ValidationPredicted.flatten
                scipy.io.savemat(DataFileName,{'Tree':Root,'Prediction':ValidationPredicted})
                if(dataset=='titanic'):
                    import save_csvTitanicRF
                    save_csvTitanicRF.results_to_csv(ValidationPredicted)
                else:
                    import save_csvSPAMDT
                    save_csvSPAMRF.results_to_csv(ValidationPredicted)
                
    
    """
    TODO: train decision tree/random forest on different datasets and perform the tasks 
    in the problem
    """

Depth 7.0
Bag 0
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
6 7.0
4 7.0
5 7.0
6 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
1 7.0
2 7.0
3 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
1
Bag 1
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
6 7.0
6 7.0
4 7.0
5 7.0
6 7.0
6 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
3 7.0
2 7.0
1
Bag 2
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
4 7.0
5 7.0
6 7.0
3 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
4 7.0
5 7.0
6 7.0
2 7.0
1
Bag 3
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
6 7.0
4 7.0
5 7.0
6 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
1 7.0
2 7.0
3 7.0
2 7.0
3 7.0
1
Bag 4
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
6 7.0
6 7.0
4 7.0
5 7.0
2 7.0
3 7.0
1
Bag 5
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
6 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
3 7.0
4 7.0
2 7.0
1
Bag 6
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
3 7.0
4 7.0
5 7.0
6 7.0


6 7.0
6 7.0
1 7.0
2 7.0
1
Bag 57
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
6 7.0
4 7.0
5 7.0
6 7.0
6 7.0
1 7.0
2 7.0
1
Bag 58
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
4 7.0
5 7.0
6 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
4 7.0
5 7.0
6 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
1
Bag 59
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
2 7.0
3 7.0
3 7.0
4 7.0
1 7.0
2 7.0
1
Bag 60
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
6 7.0
4 7.0
5 7.0
6 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
6 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
1 7.0
1
Bag 61
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
6 7.0
4 7.0
5 7.0
6 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
4 7.0
2 7.0
3 7.0
1
Bag 62
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
4 7.0
5 7.0
5 7.0
3 7.0
4 7.0
5 7.0
6 7.0
4 7.0
5 7.0
6 7.0
6 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
6 7.0
5 7.0
3 7.0
2 7.0
3 7.0
4 7.0
5 7.0
6 7.0
5 7.0
6 7.0
4 7.0
3 7.0
1
Bag 63
0 7.0
1 7.0
2 7.0
3 7.0
4 7.0
5

In [3]:
print(TrainingError,ValidationError)

[[0.21338811]] [[0.17408124]]


In [6]:
ValidationData = np.array(Z)
ValidationSize = len(Z)
ValidationPredicted = np.zeros((ValidationSize,1),dtype ='float64')
for i in range(0,ValidationSize):
    #print(i,k)
    ValidationPredicted[i] = classifier.predict(Root,ValidationData[i,:])
flat_list = [item for sublist in ValidationPredicted for item in sublist]
import save_csvSPAMRF
save_csvSPAMRF.results_to_csv(flat_list)